In [1]:
import ipyvuetify as v
from IPython.display import display, HTML
from ipywidgets import Output
import json
import os
import pandas as pd
from sqlalchemy import text
from epo.tipdata.patstat import PatstatClient
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots


# MODEL - Handles data and business logic
class VisualizationModel:
    def __init__(self, json_file='queries.json'):
        self.json_file = json_file
        self.queries = self.load_queries()
        self.patstat_client = None
        self.db = None
        self.current_df = None
    
    def initialize_patstat(self):
        """Initialize PATSTAT connection"""
        try:
            if self.patstat_client is None:
                self.patstat_client = PatstatClient()
                self.db = self.patstat_client.orm()
            return True, "PATSTAT connection initialized"
        except Exception as e:
            return False, f"Failed to initialize PATSTAT: {str(e)}"
    
    def load_queries(self):
        """Load queries from JSON file"""
        if os.path.exists(self.json_file):
            try:
                with open(self.json_file, 'r') as f:
                    data = json.load(f)
                    if isinstance(data, list):
                        return data
                    else:
                        return []
            except json.JSONDecodeError:
                return []
        else:
            return []
    
    def get_query_names(self):
        """Get list of all query names"""
        return [query['name'] for query in self.queries]
    
    def get_query_by_name(self, name):
        """Get query object by name"""
        for query in self.queries:
            if query['name'] == name:
                return query
        return None
    
    def run_query(self, query_text):
        """Execute query and return DataFrame"""
        # Initialize PATSTAT if needed
        if self.db is None:
            success, message = self.initialize_patstat()
            if not success:
                return False, message, None
        
        try:
            # Execute query
            result = self.db.execute(text(query_text))
            df = pd.DataFrame(result.fetchall(), columns=result.keys())
            
            if df.empty:
                return False, "Query returned no data", None
            
            # Store DataFrame
            self.current_df = df
            return True, f"Query executed successfully! Retrieved {len(df)} rows.", df
            
        except Exception as e:
            return False, f"Query execution failed: {str(e)}", None
    
    def create_plotly_chart(self, df):
        """Create Plotly chart from DataFrame"""
        # Find year column (case insensitive)
        year_col = None
        for col in df.columns:
            if 'year' in col.lower():
                year_col = col
                break
        
        if year_col is None:
            return None, "No 'year' column found in the data"
        
        # Find all columns with '_count' in the name
        count_cols = [col for col in df.columns if '_count' in col.lower()]
        
        if not count_cols:
            return None, "No columns with '_count' found in the data"
        
        # Create Plotly figure
        fig = go.Figure()
        
        # Add a trace for each count column
        for col in count_cols:
            # Clean up column name for legend
            legend_name = col.replace('_count', '').replace('_', ' ').title()
            
            fig.add_trace(go.Scatter(
                x=df[year_col],
                y=df[col],
                mode='lines+markers',
                name=legend_name,
                line=dict(width=2),
                marker=dict(size=6)
            ))
        
        # Update layout
        fig.update_layout(
            title='Patent Counts Over Time',
            xaxis_title=year_col.replace('_', ' ').title(),
            yaxis_title='Count',
            hovermode='x unified',
            template='plotly_white',
            legend=dict(
                orientation="v",
                yanchor="top",
                y=1,
                xanchor="left",
                x=1.02
            ),
            height=600,
            margin=dict(r=150)
        )
        
        return fig, "Chart created successfully"


# VIEW - Handles the user interface
class VisualizationView:
    def __init__(self):
        self.create_components()
        self.create_layout()
    
    def create_components(self):
        """Create all UI components"""
        # Banner
        self.banner = v.Card(
            class_='mb-4',
            elevation=3,
            children=[
                v.CardTitle(
                    class_='white--text text-h4 justify-center',
                    style_='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px;',
                    children=['Query Visualization Dashboard']
                )
            ]
        )
        
        # Query selection section
        self.selection_title = v.CardTitle(
            class_='text-h5 px-0',
            children=['📊 Select Query to Visualize']
        )
        
        # Dropdown for query selection
        self.query_dropdown = v.Select(
            label='Select Query',
            items=[],
            outlined=True,
            dense=True,
            class_='mb-3',
            v_model='',  # Initialize with empty string
            disabled=False
        )
        
        # Run button
        self.run_button = v.Btn(
            color='primary',
            children=[
                v.Icon(left=True, children=['mdi-play']),
                'Run Query'
            ],
            class_='mb-3'
        )
        
        # Status alert
        self.status_alert = v.Alert(
            type='info',
            dismissible=True,
            children=[''],
            class_='mt-3',
            style_='display: none;'
        )
        
        # Chart container with output widget
        self.chart_output = Output()
        
        self.chart_container = v.Card(
            class_='mt-4',
            elevation=2,
            style_='display: none;',
            children=[
                v.CardTitle(children=['📈 Visualization']),
                v.CardText(
                    children=[self.chart_output]
                )
            ]
        )
        
        # Info card
        self.info_card = v.Card(
            class_='mt-4',
            outlined=True,
            children=[
                v.CardText(
                    children=[
                        v.Html(tag='div', children=[
                            v.Html(tag='strong', children=['ℹ️ Instructions:']),
                            v.Html(tag='ul', children=[
                                v.Html(tag='li', children=['Select a query from the dropdown menu']),
                                v.Html(tag='li', children=['Click "Run Query" to execute and visualize']),
                                v.Html(tag='li', children=['The chart will display all count columns over time']),
                                v.Html(tag='li', children=['Requires a "year" column and columns with "_count" suffix'])
                            ])
                        ])
                    ]
                )
            ]
        )
    
    def create_layout(self):
        """Create the main layout"""
        self.layout = v.Container(
            fluid=True,
            class_='pa-4',
            style_='max-width: 1200px;',
            children=[
                self.banner,
                v.Card(
                    class_='pa-6',
                    elevation=2,
                    children=[
                        self.selection_title,
                        self.query_dropdown,
                        self.run_button,
                        self.status_alert,
                        self.chart_container,
                        self.info_card
                    ]
                )
            ]
        )
    
    def show_status(self, message, alert_type='success'):
        """Show status message"""
        self.status_alert.type = alert_type
        self.status_alert.children = [message]
        self.status_alert.style_ = ''
    
    def hide_status(self):
        """Hide status message"""
        self.status_alert.style_ = 'display: none;'
    
    def update_dropdown_items(self, items):
        """Update dropdown items"""
        self.query_dropdown.items = ['-- Select a query --'] + items
    
    def get_selected_query(self):
        """Get currently selected query name"""
        value = self.query_dropdown.v_model
        # Convert to string and handle None/bad values
        if value is None or value == '' or value == '-- Select a query --':
            return None
        return str(value)
    
    def show_chart(self, fig):
        """Display Plotly chart inside the output widget"""
        self.chart_container.style_ = ''
        # Clear previous output
        self.chart_output.clear_output(wait=True)
        # Display figure inside the output widget
        with self.chart_output:
            display(fig)
    
    def hide_chart(self):
        """Hide chart container"""
        self.chart_container.style_ = 'display: none;'
    
    def display(self):
        """Display the view"""
        display(self.layout)


# CONTROLLER - Handles user interactions and coordinates Model and View
class VisualizationController:
    def __init__(self, model, view):
        self.model = model
        self.view = view
        self.setup_event_handlers()
        self.initialize_view()
    
    def setup_event_handlers(self):
        """Set up event handlers for UI components"""
        self.view.run_button.on_event('click', self.on_run_query)
    
    def initialize_view(self):
        """Initialize the view with current data"""
        query_names = self.model.get_query_names()
        self.view.update_dropdown_items(query_names)
    
    def on_run_query(self, widget, event, data):
        """Handle run query button click"""
        selected_name = self.view.get_selected_query()
        
        # Debug: print what we got
        print(f"DEBUG: Selected query name: {repr(selected_name)}")
        print(f"DEBUG: Dropdown v_model: {repr(self.view.query_dropdown.v_model)}")
        
        if not selected_name:
            self.view.show_status('Please select a query first!', 'warning')
            return
        
        # Get query object
        query_obj = self.model.get_query_by_name(selected_name)
        
        if not query_obj:
            self.view.show_status(f'Query "{selected_name}" not found!', 'error')
            # Debug: show available queries
            print(f"DEBUG: Available queries: {self.model.get_query_names()}")
            return
        
        # Show loading message
        self.view.show_status('Running query... Please wait.', 'info')
        
        # Run query
        success, message, df = self.model.run_query(query_obj['query'])
        
        if not success:
            self.view.show_status(message, 'error')
            self.view.hide_chart()
            return
        
        # Query successful, create chart
        fig, chart_message = self.model.create_plotly_chart(df)
        
        if fig is None:
            self.view.show_status(f"Query executed but {chart_message}", 'warning')
            self.view.hide_chart()
            return
        
        # Display chart
        self.view.show_status(f'{message} {chart_message}', 'success')
        self.view.show_chart(fig)


# APPLICATION - Main entry point
class VisualizationApp:
    def __init__(self, json_file='queries.json'):
        self.model = VisualizationModel(json_file)
        self.view = VisualizationView()
        self.controller = VisualizationController(self.model, self.view)
    
    def run(self):
        """Run the application"""
        self.view.display()


# Create and run the application
if __name__ == '__main__':
    app = VisualizationApp()
    app.run()

Container(children=[Card(children=[CardTitle(children=['Query Visualization Dashboard'], class_='white--text t…